# ML Básico — Classificação de Anomalias (Industrial Sensors)

Este notebook carrega leituras de sensores, cria *features* simples e treina um classificador para identificar **anomalias** em `data/sensor_readings.csv`.

**Tarefa de ML:** classificação binária (`is_anomaly` 0/1).
**Modelo baseline:** `LogisticRegression` (rápido e interpretável).

> Requisito do desafio: usar pelo menos ~500 leituras por sensor. O repositório terá uma amostra pequena; gere a versão grande depois e reenvie para a pasta `data/`.


In [ ]:
# Se estiver no Colab, opcional: instalar dependências
# !pip -q install pandas numpy scikit-learn matplotlib

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
GRAPHS_DIR = BASE_DIR / 'graphs'
GRAPHS_DIR.mkdir(parents=True, exist_ok=True)

assert (DATA_DIR / 'sensor_readings.csv').exists(), "Coloque o dataset em data/sensor_readings.csv"


In [ ]:
# Carregar dados
readings = pd.read_csv(DATA_DIR / 'sensor_readings.csv', parse_dates=['ts'])
readings = readings.sort_values(['sensor_id', 'ts']).reset_index(drop=True)

# Filtrar leituras válidas
if 'status' in readings.columns:
    readings = readings[(readings['status'] == 'OK') | (readings['status'].isna())]
if 'quality_score' in readings.columns:
    readings = readings[(readings['quality_score'].isna()) | (readings['quality_score'] > 0)]

readings.head(10)

In [ ]:
# Feature engineering: diferenças e janelas móveis por sensor
def make_features(df: pd.DataFrame) -> pd.DataFrame:
    g = df.groupby('sensor_id', group_keys=False)
    out = df.copy()
    out['value_diff1'] = g['value'].diff(1)
    out['value_diff2'] = g['value'].diff(2)
    out['roll_mean_5'] = g['value'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)
    out['roll_std_5']  = g['value'].rolling(5, min_periods=1).std().reset_index(level=0, drop=True).fillna(0)
    out['roll_mean_15'] = g['value'].rolling(15, min_periods=1).mean().reset_index(level=0, drop=True)
    out['roll_std_15']  = g['value'].rolling(15, min_periods=1).std().reset_index(level=0, drop=True).fillna(0)
    out = out.dropna().reset_index(drop=True)
    return out

feat = make_features(readings)
target_col = 'is_anomaly'
if target_col not in feat.columns:
    raise ValueError("A coluna 'is_anomaly' não está no CSV. Adicione rótulos 0/1 para treinar.")

feature_cols = ['value','value_diff1','value_diff2','roll_mean_5','roll_std_5','roll_mean_15','roll_std_15']
X = feat[feature_cols].values
y = feat[target_col].astype(int).values
len(feat), feat.head(5)

In [ ]:
# Split, escala, treino e avaliação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_sc, y_train)

y_pred = clf.predict(X_test_sc)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))


In [ ]:
# Matriz de confusão (matplotlib puro, sem seaborn)
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(4, 4))
im = ax.imshow(cm, interpolation='nearest')
ax.set_title('Matriz de Confusão')
ax.set_xlabel('Predito')
ax.set_ylabel('Real')
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
for (i, j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha='center', va='center')
plt.tight_layout()
out_path = GRAPHS_DIR / 'confusion_matrix.png'
plt.savefig(out_path, dpi=150)
plt.show()
print(f'Figura salva em: {out_path}')


## Justificativa resumida
- **Problema:** detecção/classificação de anomalias em séries temporais de sensores.
- **Modelo:** `LogisticRegression` como baseline pela simplicidade e velocidade; pode-se testar `RandomForest` depois.
- **Features:** diferenças e estatísticas de janelas móveis capturam tendências e volatilidade.
- **Métricas:** `accuracy` + relatório de classificação; se a classe 1 for rara, priorize `F1`/`Recall`.
- **Gráfico:** matriz de confusão para visualizar acertos/erros entre classes.